Didn't miss writing a notebook for statistics that much :|

First things first we need to change the extensions of the matches to txt from tcx (the export format of choice of the garmin connect app) but first some imports

In [2]:
import os
import numpy as np
import pandas as pd
import csv
import chess
import chess.pgn
import chess.engine
from stockfish import Stockfish


In [3]:
def change_ext(cartella, old_ext, new_ext):
    for root, dirs, files in os.walk(cartella):
        # print(os.getcwd())
        for f in files:
            filename = os.fsdecode(f)
            
                
            if filename.endswith(old_ext):
                name, ext = os.path.splitext(filename)
                
                new_name = name + new_ext
                jpath = os.path.join(root,new_name)
                os.rename(os.path.join(root,filename), jpath)
                print(f"renamed {os.path.join(root,filename)} to {jpath}")

change_ext("games", ".tcx", ".txt")

Now we have to import the data from the [Portable Game Notation or pgn](https://en.wikipedia.org/wiki/Portable_Game_Notation)
and read the heartbeats of both white and black (in the games where they are both available) in order to create a 3d dataframe for every game and then to visualize the data we are working with. To process the portable game notation we will use the following cell of code.

In [4]:
### UTILS FOR READING THE ACTIVITIES

def calctime(start,curr):
    c = [int(x) for x in curr.split(":")]
    s = [int(x) for x in start.split(":")]
    return ((c[0] - s[0]) * 60 + (c[1] - s[1])) * 60 + (c[2] - s[2])

def read_activity(game):
    with open(game) as f:
        lines = f.readlines()
        hr = []
        time = []
        for s in lines:
            if "<Value>" in s:
                hr.append(s.strip()[s.strip().find(">") + 1:s.strip().rfind("<")])
            if "<Time>" in s:
                s = s.strip()[3:]
                if not time:
                    start = s.strip()[s.strip().find("T") + 1:s.strip().rfind(".")]
                    
                time.append(calctime(start, s.strip()[s.strip().find("T") + 1:s.strip().rfind(".")]))
        
        #print(f"hr is {hr}, and time is {time}")
        return list(zip(hr,time))
        
read_activity("games/partita_0525-1/b_activity_8913493582.txt")

[('60', 0),
 ('76', 1),
 ('57', 2),
 ('56', 5),
 ('57', 6),
 ('58', 12),
 ('57', 16),
 ('58', 19),
 ('59', 21),
 ('58', 22),
 ('59', 25),
 ('58', 28),
 ('57', 31),
 ('56', 38),
 ('55', 42),
 ('54', 44),
 ('53', 48),
 ('52', 52),
 ('51', 54),
 ('52', 55),
 ('51', 56),
 ('52', 58),
 ('51', 64),
 ('52', 65),
 ('53', 69),
 ('54', 72),
 ('55', 73),
 ('56', 79),
 ('55', 86),
 ('54', 91),
 ('53', 94),
 ('54', 96),
 ('55', 104),
 ('54', 105),
 ('55', 106),
 ('56', 107),
 ('58', 112),
 ('59', 116),
 ('57', 117),
 ('56', 119),
 ('57', 123),
 ('56', 129),
 ('55', 130),
 ('54', 132),
 ('55', 133),
 ('56', 135),
 ('57', 137),
 ('58', 138),
 ('57', 143),
 ('58', 145),
 ('57', 147),
 ('58', 149),
 ('59', 150),
 ('60', 152),
 ('61', 155),
 ('60', 158),
 ('59', 159),
 ('60', 161),
 ('61', 171),
 ('64', 175),
 ('63', 176),
 ('64', 178),
 ('63', 182),
 ('62', 192),
 ('61', 193),
 ('60', 194),
 ('61', 195),
 ('60', 198),
 ('61', 201),
 ('60', 205),
 ('59', 211),
 ('60', 213),
 ('61', 214),
 ('60', 216),
 

In [5]:
### UTILS FOR READING THE PGN

#this method takes as input the board and the last move so to update the board and compute the evaluation
def get_evaluation(board,move):

    stockfish = Stockfish("/usr/local/Cellar/stockfish/15/bin/stockfish")     #if no parameter passed, takes as input the executable file called stockfish from the same directory, if found
    board.push_san(move)        #insert the move in the board
    stockfish.set_fen_position(board.fen())  #bpard.fen() applies a PGN to FEN conversion
    engine = chess.engine.SimpleEngine.popen_uci("stockfish")
    info = engine.analyse(board, chess.engine.Limit(time=0.1))     #gets the infos of the game
    
    #if the position on the board is not a mate the score is in centipawns, then it's divided by 100 to get the actual score
    #Gets the score from White’s point of view
    if chess.engine.PovScore.is_mate(info['score']):
        eval = chess.engine.PovScore.white(info['score'])
    else:
        eval = int(str(chess.engine.PovScore.white(info['score'])))/100

    return board, eval      #returns the board updated and the evaluation of the position

In [6]:
def get_timestamps(file):
    timestamps = []
    file = file[:-3] + "txt"
    with open(file, "r") as f:
        file_read = f.read()
        lista = file_read.split("%")
        
        
        for el in lista:
            if el[0:4] == "time":
                num = ""
                flag = False
                for i  in range(len(el)):
                    while el[i].isdigit():
                        num += el[i]
                        i += 1
                        flag = True
                    if flag:
                        timestamps.append(int(num)/10)
                        break
    return timestamps
            
get_timestamps("/Users/tommasodimario/Documents/GitHub/statistics-101/project/src/games/partita_0816-7/tommidim_vs_roccot01_2022.08.16.txt")

[0.1,
 0.1,
 3.4,
 0.9,
 13.6,
 1.9,
 2.8,
 9.8,
 3.5,
 11.4,
 13.7,
 9.5,
 13.0,
 19.4,
 12.9,
 11.3,
 2.5,
 1.7,
 17.7,
 5.5,
 10.1,
 2.7,
 0.6,
 4.8,
 21.2,
 4.4,
 0.1,
 10.4,
 2.5,
 22.1,
 10.8,
 12.1,
 23.1,
 6.5,
 1.6,
 20.3,
 7.7,
 49.1,
 27.4,
 10.1,
 3.5,
 16.7,
 6.2,
 8.7,
 37.7,
 34.0,
 49.1,
 9.7,
 26.9,
 29.2,
 11.8,
 1.4,
 12.9,
 22.8,
 17.7,
 17.7,
 12.9,
 20.9,
 54.3,
 12.6,
 6.0,
 1.6,
 0.4,
 1.0,
 0.4,
 26.9,
 3.7,
 7.1,
 25.2,
 15.9,
 42.5,
 7.4,
 15.1,
 16.0,
 10.7,
 21.5,
 7.9,
 13.1,
 21.0,
 1.7,
 2.0,
 22.3,
 1.1,
 19.5,
 6.0,
 18.0,
 19.3,
 12.7,
 0.7,
 3.9,
 2.0,
 6.2,
 3.8,
 1.3,
 3.1,
 3.2,
 7.7,
 1.5,
 0.1,
 1.1]

In [7]:
#read_pgn takes as input the name of the file pgn containing the game and returns a dataframe
def read_pgn(game_name): 
    old = False
    if "05.29" in game_name or "05.25" in game_name or "clk" in game_name:
        old = True
    #opens and reads the pgn file
    if not old:
        timestamps = get_timestamps(game_name)
    pgn = open(game_name) 
    
    game = chess.pgn.read_game(pgn)
    
                                   
    array = []      #initialize the array that will contain the data
    index_values = [] 
    i=0
    time_white = -0.1      #to keep track of the time for each player
    time_black = 0      #to keep track of the time for each player
    board = chess.Board()       #initialize the chessboard

    #iterate through the moves
    for node in game.mainline():        
        
        move = str(node.move)       #represents the last move
        board,evaluation = get_evaluation(board,move)       #the get_evaluation method returns the updated board and the evaluation of the current position
        
        
        if i%2 == 0:        #if the white is moving we update his time
            index_values.append("white")
            if old:
                time_white = 600 - node.clock()
            else:
                time_white += timestamps[i]
                array.append([str(node.move),round(time_white,4),round(time_white+time_black,4),evaluation])
            
           
        else:           #if the black is moving we update his time
            index_values.append("black") 
            if old:
                time_black = 600 - node.clock()
            else:
                time_black += timestamps[i]
                array.append([str(node.move),round(time_black,4),round(time_white+time_black,4),evaluation])

        #evaluation = get_evaluation(board,move)
        #we append an element in the array (a row in the dataframe) with move, time the player used, total time since the game started and evaluation of the position
        # if str(evaluation)[:2] == "#+":
        #     evaluation = 15
        # elif str(evaluation)[:2] == "#-":
        #     evaluation = -15
        if old:
            array.append([str(node.move),round(600 - node.clock(),4),round(time_white+time_black,4),evaluation])   
        i+=1
    
    column_values = ['move','time','real time','evaluation'] #create the columns names
    array = array      #convert our array to a numpy array
    res = []
    j = 0
    #df = pd.DataFrame(data = array, index = index_values, columns = column_values) 
    
    tub = 0
    tuw = 0
    for i in range(int(array[-1][2]) + 1):
        mosse = []
        
        while float(array[j][2]) <= float(i):
            pos = array[j][3]
            mosse.append(array[j][0])
            if j % 2 == 0:
                tuw = array[j][1]
            elif j % 2 != 0:
                tub = array[j][1]
            j += 1
            if j > len(array):
                break
        res.append([i, pos, (tuw,tub), mosse])
        
            
        
    #create a dataframe
    print(array)
    return res


res = read_pgn("/Users/tommasodimario/Documents/GitHub/statistics-101/project/src/games/partite_confronto/partita_timestamp.pgn")   
res

#/Users/tommasodimario/Documents/GitHub/statistics-101/project/src/games/partite_confronto/partita_timestamp.pgn

[['e2e4', 0.0, 0.0, 0.38], ['e7e5', 2.2, 2.2, 0.42], ['g1f3', 1.3, 3.5, 0.37], ['b8c6', 4.6, 5.9, 0.62], ['b1c3', 2.2, 6.8, 0.21], ['f8b4', 13.2, 15.4, 0.69], ['a2a3', 12.5, 25.7, 0.08], ['b4c3', 16.5, 29.0, 0.15], ['d2c3', 14.7, 31.2, 0.12], ['g8f6', 24.1, 38.8, 0.03], ['f1d3', 23.5, 47.6, 0.1], ['d7d5', 38.8, 62.3, 0.17], ['e4d5', 55.9, 94.7, -0.12], ['d8d5', 43.5, 99.4, -0.12], ['e1g1', 68.5, 112.0, -0.17], ['e5e4', 44.9, 113.4, 2.99], ['f1e1', 83.3, 128.2, -5.25], ['e8g8', 56.0, 139.3, -5.1], ['d3e4', 87.9, 143.9, -5.82], ['f6e4', 58.4, 146.3, 9.35], ['d1d5', 101.9, 160.3, 9.82]]


[[0, 0.38, (0.0, 0), ['e2e4']],
 [1, 0.38, (0.0, 0), []],
 [2, 0.38, (0.0, 0), []],
 [3, 0.42, (0.0, 2.2), ['e7e5']],
 [4, 0.37, (1.3, 2.2), ['g1f3']],
 [5, 0.37, (1.3, 2.2), []],
 [6, 0.62, (1.3, 4.6), ['b8c6']],
 [7, 0.21, (2.2, 4.6), ['b1c3']],
 [8, 0.21, (2.2, 4.6), []],
 [9, 0.21, (2.2, 4.6), []],
 [10, 0.21, (2.2, 4.6), []],
 [11, 0.21, (2.2, 4.6), []],
 [12, 0.21, (2.2, 4.6), []],
 [13, 0.21, (2.2, 4.6), []],
 [14, 0.21, (2.2, 4.6), []],
 [15, 0.21, (2.2, 4.6), []],
 [16, 0.69, (2.2, 13.2), ['f8b4']],
 [17, 0.69, (2.2, 13.2), []],
 [18, 0.69, (2.2, 13.2), []],
 [19, 0.69, (2.2, 13.2), []],
 [20, 0.69, (2.2, 13.2), []],
 [21, 0.69, (2.2, 13.2), []],
 [22, 0.69, (2.2, 13.2), []],
 [23, 0.69, (2.2, 13.2), []],
 [24, 0.69, (2.2, 13.2), []],
 [25, 0.69, (2.2, 13.2), []],
 [26, 0.08, (12.5, 13.2), ['a2a3']],
 [27, 0.08, (12.5, 13.2), []],
 [28, 0.08, (12.5, 13.2), []],
 [29, 0.15, (12.5, 16.5), ['b4c3']],
 [30, 0.15, (12.5, 16.5), []],
 [31, 0.15, (12.5, 16.5), []],
 [32, 0.12, (14.7,

In [20]:
file = open("prova.csv", "w")

writer = csv.writer(file)

columns = ["Time", "Game", "Player", "Statistics", "Value"]
writer.writerow(columns)

#pseudo code
# to do create queste cose così da poter iterare in questa roba della morte e creare un dataset di questo tipo
# https://raw.githubusercontent.com/QuantEcon/lecture-python/master/source/_static/lecture_specific/pandas_panel/realwage.csv
# sembra particolarmente greve ma se n è il numero di statistiche gira in ~20*2*3 * n quindi è un n^4 molto tranquillo
print(os.getcwd())



gamecount = 0
count = 0   
for root,dirs, files in os.walk("games"):
    if str(root)  != "games":
        partita = f"Game{gamecount}"
        gamecount += 1
        if gamecount > 1: # così fa solo le prime tre partite
            break
    for file in files:
        #qui siamo dentro i file di una singola partita
        filename = os.fsdecode(file)
        print(filename)
        if filename.endswith(".txt") and (filename[0] == "w" or filename[0] == "b"): # abbiamo un carissimo file delle attività
        #leggiamo i battiti
            player = filename[0]
            activity = read_activity(f"{str(root)}/{filename}")
            # As explained in the markdown above on our strumentation TODO our smartwatches recorded the heartbeats at different time so we smoothed it out
            for hr, time in activity:
                row = [count,int(time), partita, player, "HR", int(hr)]
                writer.writerow(row)
                count +=1
        elif filename.endswith(".pgn"):
            
            pgn = read_pgn(f"{str(root)}/{filename}")
            for second in range(len(pgn)):
                ev = pgn[second][1]
                try:
                    ev = float(pgn[second][1])
                except:
                    ev = 20
                row = [count,second, partita, "White", "Evaluation", ev]
                
                writer.writerow(row)
                count +=1
                    
            for second in range(len(pgn)):
                for player in ["Black", "White"]:       
                    p = 0 if player == "White" else 1
                    row = [count, second, partita, player, "TimeUsed", int(pgn[second][2][p])]
                    writer.writerow(row)
                    count +=1
# 4.



/Users/tommasodimario/Documents/GitHub/statistics-101/project/src
.DS_Store
.DS_Store
roccot01_vs_tommidim_2022.08.16.txt
b_activity_9418334461_tom.txt
roccot01_vs_tommidim_2022.08.16.pgn
[['e2e4', 0.0, 0.0, 0.38], ['e7e5', 0.1, 0.1, 0.55], ['b1c3', 11.1, 11.2, 0.16], ['g8f6', 5.4, 16.5, 0.41], ['f1c4', 19.2, 24.6, 0.09], ['c7c6', 35.4, 54.6, 0.86], ['g1f3', 50.2, 85.6, 0.01], ['d7d5', 41.0, 91.2, 0.41], ['e4d5', 71.8, 112.8, 0.0], ['c6d5', 42.2, 114.0, 1.01], ['c4b5', 85.1, 127.3, 0.71], ['c8d7', 51.6, 136.7, 0.79], ['f3e5', 89.7, 141.3, 0.83], ['d7b5', 59.0, 148.7, 1.21], ['c3b5', 95.4, 154.4, 1.11], ['d8e7', 60.9, 156.3, 1.81], ['d1e2', 120.9, 181.8, 2.37], ['b8c6', 154.5, 275.4, 5.21], ['d2d4', 131.6, 286.1, 1.85], ['a7a6', 159.3, 290.9, 1.77], ['e5c6', 220.1, 379.4, 1.81], ['e7e2', 213.4, 433.5, 1.78], ['e1e2', 222.1, 435.5, 1.73], ['a6b5', 214.8, 436.9, 1.72], ['c6e5', 235.8, 450.6, 1.61], ['f8d6', 226.4, 462.2, 2.16], ['h1e1', 242.2, 468.6, 1.77], ['e8g8', 229.4, 471.6, 1.74], [

In [21]:

# Display 6 columns for viewing purposes
pd.set_option('display.max_columns', 10)

# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

realwage = pd.read_csv("prova.csv")
realwage = realwage.pivot_table(values='Value',
                                index='Time',
                                columns=['Game', 'Player', 'Statistics'])
realwage.head()

Game          Game0                                       
Player        Black      White                     b     w
Statistics TimeUsed Evaluation    HR TimeUsed     HR    HR
Time                                                      
0              0.00       0.38 75.00     0.00  88.00 75.00
1              0.00       0.55 93.00     0.00 103.00 93.00
2              0.00       0.55   nan     0.00  86.00   nan
3              0.00       0.55   nan     0.00  87.00   nan
4              0.00       0.55   nan     0.00  87.00   nan

In [10]:
realwage["Game0"]

Player        Black      White         
Statistics TimeUsed Evaluation TimeUsed
Time                                   
0              0.00       0.38     0.00
1              0.00       0.38     0.00
2              0.00       0.38     0.00
3              2.00       0.55     0.00
4              2.00       0.37     1.00
...             ...        ...      ...
156           58.00       9.35    87.00
157           58.00       9.35    87.00
158           58.00       9.35    87.00
159           58.00       9.35    87.00
160           58.00       9.35    87.00

[161 rows x 3 columns]